# Importando

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('H:\Ambiente de Trabalho\JEST\Projeto\ProjetoTI_part2-master\Leukemya_data.csv', header = None)
target = pd.read_csv('H:\Ambiente de Trabalho\JEST\Projeto\ProjetoTI_part2-master\labels.csv', header = None)

# Alterando valores para 0 e 1 na tabela de labels

In [4]:
y = target.applymap(lambda n: n-1)

# Dividir dados para treinamento

In [5]:
df_train = df.head(128)
df_test = df.tail(50)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df_train, test_size=0.3, shuffle=False, stratify=None)
y_train, y_test = train_test_split(y, test_size=0.3, shuffle=False, stratify=None)

# Importando

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix

In [8]:
def evaluationModels(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    # Best metric for binary (0/1) classification 
    mcc = matthews_corrcoef(y_test, y_pred)
    FN = confusion_matrix(y_test, y_pred)[0][1]
    return accuracy, precision, mcc, FN


def printMetrics(model, accuracy, precision, mcc, fp):
    modelName = model.__class__.__name__
    bar = "-" * 25
    print(f"{bar}\n {modelName}\n{bar}")
    print(f'- Accuracy: {accuracy:0.3f}')
    print(f'- Precision: {precision:0.3f}')
    print(f'- MCC: {mcc:0.3f}')
    print(f'- False Positives: {fp}')

# Decision Tree Classifier

In [9]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier(max_depth=3, random_state=42)
DT.fit(X_train, y_train)
y_pred_DT = DT.predict(X_test)

In [10]:
accuracy, precision, mcc, fp = metricsDT = evaluationModels(y_test, y_pred_DT)
printMetrics(DT, accuracy, precision, mcc, fp)

-------------------------
 DecisionTreeClassifier
-------------------------
- Accuracy: 0.923
- Precision: 0.833
- MCC: 0.726
- False Positives: 1


# Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(max_depth=4, random_state=42)
RF.fit(X_train, y_train)
y_pred_RF = RF.predict(X_test)

C:\Users\TESTER\AppData\Local\Temp/ipykernel_9860/3898969665.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF.fit(X_train, y_train)


In [12]:
accuracy, precision, mcc, fp = metricsRF = evaluationModels(y_test, y_pred_RF)
printMetrics(RF, accuracy, precision, mcc, fp)

-------------------------
 RandomForestClassifier
-------------------------
- Accuracy: 0.949
- Precision: 1.000
- MCC: 0.820
- False Positives: 0


# K Nearest Neighbors (KNN)

In [13]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(metric='euclidean')
KNN.fit(X_train, y_train)
y_pred_KNN = KNN.predict(X_test)

H:\Programas Faculdade\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [14]:
accuracy, precision, mcc, fp = metricsKNN = evaluationModels(y_test, y_pred_KNN)
printMetrics(KNN, accuracy, precision, mcc, fp)

-------------------------
 KNeighborsClassifier
-------------------------
- Accuracy: 0.872
- Precision: 1.000
- MCC: 0.497
- False Positives: 0


# XGBoost (Ensemble model)

In [15]:
import xgboost as xgb

XGB = xgb.XGBClassifier(max_depth=4, random_state=42)
XGB.fit(X_train, y_train)
y_pred_XGB = XGB.predict(X_test)

ModuleNotFoundError: No module named 'xgboost'

In [16]:
accuracy, precision, mcc, fp = metricsXGB = evaluationModels(y_test, y_pred_XGB)
printMetrics(XGB, accuracy, precision, mcc, fp)

NameError: name 'y_pred_XGB' is not defined

# Neural Network

In [17]:
from sklearn.neural_network import MLPClassifier

MLP = MLPClassifier(alpha=1, max_iter=1000)
MLP.fit(X_train, y_train)
y_pred_MLP = MLP.predict(X_test)

H:\Programas Faculdade\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [18]:
accuracy, precision, mcc, fp = metricsMLP = evaluationModels(y_test, y_pred_MLP)
printMetrics(MLP, accuracy, precision, mcc, fp)

-------------------------
 MLPClassifier
-------------------------
- Accuracy: 0.923
- Precision: 1.000
- MCC: 0.723
- False Positives: 0


# Comparison of models

In [19]:
results_accuracy = {
    'Threshold': metricsThreshold[0],
    'DT': metricsDT[0],
    'RF': metricsRF[0],
    'KNN': metricsKNN[0],
    'MLP': metricsMLP[0],
    'XGB': metricsXGB[0],
    
}
results_precision = {
    'Threshold': metricsThreshold[1],
    'DT': metricsDT[1],
    'RF': metricsRF[1],
    'KNN': metricsKNN[1],
    'MLP': metricsMLP[1],
    'XGB': metricsXGB[1],
}
results_mcc = {
    'Threshold': metricsThreshold[2],
    'DT': metricsDT[2],
    'RF': metricsRF[2],
    'KNN': metricsKNN[2],
    'MLP': metricsMLP[2],
    'XGB': metricsXGB[2],
}

results_fp = {
    'Threshold': metricsThreshold[3],
    'DT': metricsDT[3],
    'RF': metricsRF[3],
    'KNN': metricsKNN[3],
    'MLP': metricsMLP[3],
    'XGB': metricsXGB[3],
}

acc_df = pd.DataFrame.from_dict(results_accuracy, orient='index', columns=['Accuracy'])
precision_df = pd.DataFrame.from_dict(results_precision, orient='index', columns=['Precision'])
mcc_df = pd.DataFrame.from_dict(results_mcc, orient='index', columns=['MCC'])
fp_df = pd.DataFrame.from_dict(results_fp, orient='index', columns=['FP'])
results = pd.concat([acc_df, precision_df, mcc_df, fp_df], axis=1)
results

NameError: name 'metricsThreshold' is not defined